In [137]:
! pip3 install pyspark seaborn pandas

import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
import seaborn as sns

import sys
from pyspark.sql import SparkSession
import argparse
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType



[notice] A new release of pip available: 22.3.1 -> 23.1
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [139]:
customers_df = pd.read_csv('./data/customers.csv')
articles_df = pd.read_csv('./data/articles.csv')
transactions_df = pd.read_csv('./data/transactions_train.csv')

### Explore

In [ ]:
articles_df.isnull().sum()


article_id                        0
product_code                      0
prod_name                         0
product_type_no                   0
product_type_name                 0
product_group_name                0
graphical_appearance_no           0
graphical_appearance_name         0
colour_group_code                 0
colour_group_name                 0
perceived_colour_value_id         0
perceived_colour_value_name       0
perceived_colour_master_id        0
perceived_colour_master_name      0
department_no                     0
department_name                   0
index_code                        0
index_name                        0
index_group_no                    0
index_group_name                  0
section_no                        0
section_name                      0
garment_group_no                  0
garment_group_name                0
detail_desc                     416
dtype: int64

In [ ]:
transactions_df.isnull().sum()


t_dat               0
customer_id         0
article_id          0
price               0
sales_channel_id    0
dtype: int64

In [ ]:
# print(customers_df.duplicated().sum())
# print(articles_df.duplicated().sum())
# print(transactions_df.duplicated().sum())
transactions_df[transactions_df.duplicated()].head()

,t_dat,customer_id,article_id,price,sales_channel_id
15,2018-09-20,000aa7f0dc06cd7174389e76c9e132a67860c5f65f9706...,501820043,0.016932,2
18,2018-09-20,000aa7f0dc06cd7174389e76c9e132a67860c5f65f9706...,671505001,0.033881,2
20,2018-09-20,000aa7f0dc06cd7174389e76c9e132a67860c5f65f9706...,631848002,0.033881,2
21,2018-09-20,000aa7f0dc06cd7174389e76c9e132a67860c5f65f9706...,631848002,0.033881,2
22,2018-09-20,000aa7f0dc06cd7174389e76c9e132a67860c5f65f9706...,631848002,0.033881,2


In [ ]:
transactions_df.count()


t_dat               31788324
customer_id         31788324
article_id          31788324
price               31788324
sales_channel_id    31788324
dtype: int64

In [ ]:
transactions_df = transactions_df.drop_duplicates()

In [ ]:
transactions_df.count()

t_dat               28813419
customer_id         28813419
article_id          28813419
price               28813419
sales_channel_id    28813419
dtype: int64

In [ ]:
transactions_df.head()

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2


### Deduplication

#### Articles

##### Several data points are represented as both nominal and discrete values, 
* Retain only nominal data for processing such as product_type_name, graphical_appearance_name
* Drop discrete data value such as product_type_no, graphical_appearance_no

In [140]:
redundant_columns = [
    'product_code',
    'product_type_no',
    'graphical_appearance_no',
    'colour_group_code',
    'perceived_colour_value_id',
    'perceived_colour_master_id',
    'department_no',
    'index_code',
    'index_group_no',
    'section_no',
    'garment_group_no'
]

articles_df.drop(columns = redundant_columns, axis=1, inplace = True)

##### Article categories can be represetned solely using either index_name or index_group_name
* index_name provides further subcategories
* drop index_group_names

In [141]:
articles_df.groupby(['index_group_name', 'index_name']).count()['article_id']

index_group_name  index_name                    
Baby/Children     Baby Sizes 50-98                   8875
                  Children Accessories, Swimwear     4615
                  Children Sizes 134-170             9214
                  Children Sizes 92-140             12007
Divided           Divided                           15149
Ladieswear        Ladies Accessories                 6961
                  Ladieswear                        26001
                  Lingeries/Tights                   6775
Menswear          Menswear                          12553
Sport             Sport                              3392
Name: article_id, dtype: int64

In [142]:
articles_df.drop("index_group_name", axis=1, inplace = True)

### Missing Values

#### Articles

##### Items where the percieved colour, graphical appearance and colour groups are unknown cannot be properly identified as trending
* Drop articles where key characteristic data is `Unknown`,
    * perceived_colour_value_name is `Unknown`
    * graphical_appearance_name is `Unknown`
    * colour_group_name is `Unknown`
    * perceived_colour_master_name is `Unknown`

In [143]:
df = articles_df.loc[(articles_df['perceived_colour_value_name'] == 'Unknown')]
df.groupby(['perceived_colour_value_name', 'perceived_colour_master_name', 'colour_group_name','graphical_appearance_name']).count()['article_id']

perceived_colour_value_name  perceived_colour_master_name  colour_group_name  graphical_appearance_name
Unknown                      Unknown                       Unknown            Unknown                      28
Name: article_id, dtype: int64

In [144]:
articles_df.drop(articles_df.loc[articles_df['perceived_colour_value_name'] == 'Unknown'].index, inplace=True, axis=0)

In [145]:
articles_df.head()

,article_id,prod_name,product_type_name,product_group_name,graphical_appearance_name,colour_group_name,perceived_colour_value_name,perceived_colour_master_name,department_name,index_name,section_name,garment_group_name,detail_desc
0,108775015,Strap top,Vest top,Garment Upper body,Solid,Black,Dark,Black,Jersey Basic,Ladieswear,Womens Everyday Basics,Jersey Basic,Jersey top with narrow shoulder straps.
1,108775044,Strap top,Vest top,Garment Upper body,Solid,White,Light,White,Jersey Basic,Ladieswear,Womens Everyday Basics,Jersey Basic,Jersey top with narrow shoulder straps.
2,108775051,Strap top (1),Vest top,Garment Upper body,Stripe,Off White,Dusty Light,White,Jersey Basic,Ladieswear,Womens Everyday Basics,Jersey Basic,Jersey top with narrow shoulder straps.
3,110065001,OP T-shirt (Idro),Bra,Underwear,Solid,Black,Dark,Black,Clean Lingerie,Lingeries/Tights,Womens Lingerie,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
4,110065002,OP T-shirt (Idro),Bra,Underwear,Solid,White,Light,White,Clean Lingerie,Lingeries/Tights,Womens Lingerie,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."


In [97]:
customers_df.head()

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,NaN,NaN,ACTIVE,NONE,49.0,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,NaN,NaN,ACTIVE,NONE,25.0,2973abc54daa8a5f8ccfe9362140c63247c5eee03f1d93...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,NaN,NaN,ACTIVE,NONE,24.0,64f17e6a330a85798e4998f62d0930d14db8db1c054af6...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,NaN,NaN,ACTIVE,NONE,54.0,5d36574f52495e81f019b680c843c443bd343d5ca5b1c2...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,1.0,1.0,ACTIVE,Regularly,52.0,25fa5ddee9aac01b35208d01736e57942317d756b32ddd...


In [98]:
customers_df.isnull().sum()

customer_id                    0
FN                        895050
Active                    907576
club_member_status          6062
fashion_news_frequency     16009
age                        15861
postal_code                    0
dtype: int64

In [99]:
customers_df.describe()

,FN,Active,age
count,476930.0,464404.0,1.356119e+06
mean,1.0,1.0,3.638696e+01
std,0.0,0.0,1.431363e+01
min,1.0,1.0,1.600000e+01
25%,1.0,1.0,2.400000e+01
50%,1.0,1.0,3.200000e+01
75%,1.0,1.0,4.900000e+01
max,1.0,1.0,9.900000e+01


In [100]:
for key in ["club_member_status", "fashion_news_frequency"]:
    for val in customers_df[key].unique():
        print(f"{key}={val}")
        display(customers_df[customers_df[key] == val])

club_member_status=ACTIVE


,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,NaN,NaN,ACTIVE,NONE,49.0,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,NaN,NaN,ACTIVE,NONE,25.0,2973abc54daa8a5f8ccfe9362140c63247c5eee03f1d93...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,NaN,NaN,ACTIVE,NONE,24.0,64f17e6a330a85798e4998f62d0930d14db8db1c054af6...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,NaN,NaN,ACTIVE,NONE,54.0,5d36574f52495e81f019b680c843c443bd343d5ca5b1c2...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,1.0,1.0,ACTIVE,Regularly,52.0,25fa5ddee9aac01b35208d01736e57942317d756b32ddd...
...,...,...,...,...,...,...,...
1371974,ffffaff3905b803d1c7e153a1378a5151e1f34f236ba54...,1.0,1.0,ACTIVE,Regularly,21.0,2c29ae653a9282cce4151bd87643c907644e09541abc28...
1371975,ffffbbf78b6eaac697a8a5dfbfd2bfa8113ee5b403e474...,NaN,NaN,ACTIVE,NONE,24.0,7aa399f7e669990daba2d92c577b52237380662f36480b...
1371976,ffffcd5046a6143d29a04fb8c424ce494a76e5cdf4fab5...,NaN,NaN,ACTIVE,NONE,21.0,3f47f1279beb72215f4de557d950e0bfa73789d24acb5e...
1371977,ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1...,1.0,1.0,ACTIVE,Regularly,21.0,4563fc79215672cd6a863f2b4bf56b8f898f2d96ed590e...


club_member_status=nan


,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code


club_member_status=PRE-CREATE


...

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code


fashion_news_frequency=Monthly


,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
5040,00f38d434368a5872c481d64d1f3b2289f3ed4d2cb5d99...,1.0,1.0,ACTIVE,Monthly,48.0,fed79dd607669cb0b9205ef909f87ec433f2c49a44f41b...
6412,01318a0ceef3dd69af82d134a96b6b00ee73693bf72cab...,1.0,1.0,ACTIVE,Monthly,34.0,e7d38730139c0bcbe836cb017cc0423b533b6db34c6807...
6523,0136c28ff3d96e5a5ef25cf5281114d80ab51fc89987eb...,1.0,1.0,ACTIVE,Monthly,34.0,991c917260c114d30d7c8ed76ac4306ef0c9f8d5205911...
6560,01387afda07032df0f33f86bdbd1b09df918241ef7db9b...,1.0,1.0,ACTIVE,Monthly,62.0,114f0d0069a2a2cccfe7b72e917d46412b33103c494742...
9771,01cecf8865bc39b0a7c8af386c2aa0cd7c3fc18d61f344...,1.0,1.0,ACTIVE,Monthly,27.0,f680358c8948524daef52657d1d6f791eb0e4b0e057290...
...,...,...,...,...,...,...,...
1366503,fef5935e8f60e59874ddc8692172b6ac0d61c6dc4241a6...,1.0,1.0,ACTIVE,Monthly,46.0,42f5025eb7aa591c5ae52dbbe2e3cbf6bab47e029a3848...
1366979,ff0c9eab0a7e15a3aa5d541dce7a5ab2bc947f69c8ff00...,1.0,1.0,ACTIVE,Monthly,30.0,5a1f05bfffbc43cf1f875a353cd7e2d8c8794fe2752d08...
1367297,ff1b951242dfefe9ff85631499c425d7ba7eb4e164a1ac...,1.0,1.0,ACTIVE,Monthly,30.0,6ab8da56290ff3d778e9d9e98dd0a0feedeb688664aa9c...
1370406,ffb3d2f2b9b3c371e91acdf8438a5306dc04989e7b6192...,1.0,1.0,ACTIVE,Monthly,26.0,0c264688d9d09cc4f8a797d068fd39bb066b8154f146d4...


fashion_news_frequency=None


,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
898114,a79d9cbfaceb0d25a91caccfad167d4d6391fd5bb4292b...,1.0,NaN,ACTIVE,None,38.0,58b7c5c27f8b5e8118a9786379601a344ab2e141550d93...
1356686,fd2371195b3d046afbb75fb0b9b6a3386275aaae19cd5a...,1.0,1.0,ACTIVE,None,64.0,170a318776b1acf12fb92fe25eb39ff88fd363b530573e...


In [101]:
customers_map_df = customers_df.copy()
customers_map_df["club_member_status"] = customers_map_df["club_member_status"].map({
    "ACTIVE": "active",
    "LEFT CLUB": "left_club",
    "PRE-CREATE": "precreate"
})
customers_map_df["fashion_news_frequency"] = customers_map_df["fashion_news_frequency"].map({
    "Regularly": "regularly",
    "Monthly": "monthly"
})
customer_transformer = ColumnTransformer(
    transformers = [
        (
            'encode_categorical',
            OneHotEncoder(),
            ["club_member_status", "fashion_news_frequency"]
        ),
        (
            'impute_age',
            SimpleImputer(),
            ["age"]
        ),
        (
            'passthrough_metadata',
            'passthrough',
            ["customer_id", "postal_code"]
        ),
        (
            'drop_misc',
            'drop',
            ["FN", "Active"]
        )
    ],
    sparse_threshold=0,
    verbose_feature_names_out=False
).fit(customers_map_df)

transformed_customers_df = pd.DataFrame(customer_transformer.transform(customers_map_df), columns=customer_transformer.get_feature_names_out())
display(transformed_customers_df.isnull().sum())
display(transformed_customers_df.describe())
transformed_customers_df

club_member_status_active           0
club_member_status_left_club        0
club_member_status_precreate        0
club_member_status_nan              0
fashion_news_frequency_monthly      0
fashion_news_frequency_regularly    0
fashion_news_frequency_nan          0
age                                 0
customer_id                         0
postal_code                         0
dtype: int64

,club_member_status_active,club_member_status_left_club,club_member_status_precreate,club_member_status_nan,fashion_news_frequency_monthly,fashion_news_frequency_regularly,fashion_news_frequency_nan,age,customer_id,postal_code
count,1371980.0,1371980.0,1371980.0,1371980.0,1371980.0,1371980.0,1371980.0,1371980.0,1371980,1371980
unique,2.0,2.0,2.0,2.0,2.0,2.0,2.0,85.0,1371980,352899
top,1.0,0.0,0.0,0.0,0.0,0.0,1.0,21.0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,2c29ae653a9282cce4151bd87643c907644e09541abc28...
freq,1272491.0,1371513.0,1279020.0,1365918.0,1371138.0,894564.0,893722.0,67530.0,1,120303


,club_member_status_active,club_member_status_left_club,club_member_status_precreate,club_member_status_nan,fashion_news_frequency_monthly,fashion_news_frequency_regularly,fashion_news_frequency_nan,age,customer_id,postal_code
0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,49.0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...
1,1.0,0.0,0.0,0.0,0.0,0.0,1.0,25.0,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,2973abc54daa8a5f8ccfe9362140c63247c5eee03f1d93...
2,1.0,0.0,0.0,0.0,0.0,0.0,1.0,24.0,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,64f17e6a330a85798e4998f62d0930d14db8db1c054af6...
3,1.0,0.0,0.0,0.0,0.0,0.0,1.0,54.0,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,5d36574f52495e81f019b680c843c443bd343d5ca5b1c2...
4,1.0,0.0,0.0,0.0,0.0,1.0,0.0,52.0,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,25fa5ddee9aac01b35208d01736e57942317d756b32ddd...
...,...,...,...,...,...,...,...,...,...,...
1371975,1.0,0.0,0.0,0.0,0.0,0.0,1.0,24.0,ffffbbf78b6eaac697a8a5dfbfd2bfa8113ee5b403e474...,7aa399f7e669990daba2d92c577b52237380662f36480b...
1371976,1.0,0.0,0.0,0.0,0.0,0.0,1.0,21.0,ffffcd5046a6143d29a04fb8c424ce494a76e5cdf4fab5...,3f47f1279beb72215f4de557d950e0bfa73789d24acb5e...
1371977,1.0,0.0,0.0,0.0,0.0,1.0,0.0,21.0,ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1...,4563fc79215672cd6a863f2b4bf56b8f898f2d96ed590e...
1371978,1.0,0.0,0.0,0.0,0.0,1.0,0.0,18.0,ffffd7744cebcf3aca44ae7049d2a94b87074c3d4ffe38...,8892c18e9bc3dca6aa4000cb8094fc4b51ee8db2ed14d7...


### Spark Join

In [ ]:
# TESTING PURPOSES ONLY
# Perform joins on samples and verify output
spark = SparkSession.builder.getOrCreate()

sample_cust_spark_df = spark.read.format("csv").option("header", "true").load("./spark_test_data/cust_sample.csv")
sample_art_spark_df = spark.read.format("csv").option("header", "true").load("./spark_test_data/articles_sample.csv")
sample_txn_spark_df = spark.read.format("csv").option("header", "true").load("./spark_test_data/txn_sample.csv")

joined_customer_transactions_df = sample_txn_spark_df.join(sample_cust_spark_df, on="customer_id")
joined_sample_df = joined_customer_transactions_df.join(sample_art_spark_df, on=['article_id'], how='inner')

joined_sample_df.write.csv("./spark_test_data/output", mode='overwrite', header=True)

In [ ]:
# Perform joins on full df
spark = SparkSession.builder.getOrCreate()

spark_customer_df = spark.read.format("csv").option("header", "true").load("./data/customers.csv")
spark_articles_df = spark.read.format("csv").option("header", "true").load("./data/articles.csv")
spark_transactions_df = spark.read.format("csv").option("header", "true").load("./data/transactions_train.csv")

joined_cust_trans_df = spark_transactions_df.join(spark_customer_df, on="customer_id")
joined_df = joined_cust_trans_df.join(spark_articles_df, on=['article_id'], how='inner')

In [24]:
joined_df.coalesce(1).write.csv("./data/joined_dataframe", mode='overwrite', header=True)

In [26]:
print(joined_df.head(1))

[Row(article_id='0147339034', customer_id='e8769150304d10862f5d4de22e461a4c6e435bd4dcc58fa146949d5e52638c88', t_dat='2018-12-14', price='0.03049152542372881', sales_channel_id='2', FN='1.0', Active='1.0', club_member_status='ACTIVE', fashion_news_frequency='Regularly', age='66', postal_code='9b24635b57ae2535445a00d7534066aa385a17317616d15d61da8527341dddf5', product_code='0147339', prod_name='6P SS BODY', product_type_no='256', product_type_name='Bodysuit', product_group_name='Garment Upper body', graphical_appearance_no='1010001', graphical_appearance_name='All over pattern', colour_group_code='10', colour_group_name='White', perceived_colour_value_id='3', perceived_colour_value_name='Light', perceived_colour_master_id='9', perceived_colour_master_name='White', department_no='6515', department_name='Baby basics', index_code='G', index_name='Baby Sizes 50-98', index_group_no='4', index_group_name='Baby/Children', section_no='44', section_name='Baby Essentials & Complements', garment_gro

In [27]:
print(joined_df.columns)

['article_id', 'customer_id', 't_dat', 'price', 'sales_channel_id', 'FN', 'Active', 'club_member_status', 'fashion_news_frequency', 'age', 'postal_code', 'product_code', 'prod_name', 'product_type_no', 'product_type_name', 'product_group_name', 'graphical_appearance_no', 'graphical_appearance_name', 'colour_group_code', 'colour_group_name', 'perceived_colour_value_id', 'perceived_colour_value_name', 'perceived_colour_master_id', 'perceived_colour_master_name', 'department_no', 'department_name', 'index_code', 'index_name', 'index_group_no', 'index_group_name', 'section_no', 'section_name', 'garment_group_no', 'garment_group_name', 'detail_desc']
